In [1]:
from mnmt.encoder import BasicEncoder
from mnmt.decoder import BasicFeedForwardDecoder
from mnmt.decoder import GreedyDecoder
from mnmt.attention import AdditiveAttention
from mnmt.decoder import BridgeLayer
from mnmt.model import Seq2Seq
from mnmt.datasets import *
from mnmt.inputter import ArgsFeeder
from mnmt.inputter import ModuleArgsFeeder
from mnmt.trainer.utils import *
from mnmt.trainer import Trainer
import sys

Loading DICT dataset...
Loading the Dataset into the container...
Field names: ['en', 'ch', 'pinyin_str', 'pinyin_char']
Data sizes: [(Train, 46620), (Valid, 5828), (Test, 5828)]
Loading NEWS dataset...
Loading the Dataset into the container...
Field names: ['en', 'ch', 'pinyin_str', 'pinyin_char']
Data sizes: [(Train, 81252), (Valid, 513), (Test, 1000)]


In [2]:
def set_up_args(data_container, exp_num):
    build_vocabs(data_container, dict_min_freqs={'en': 1, 'ch': 1, 'pinyin_str': 1, 'pinyin_char': 1})
    for name, field in data_container.fields:
        if name == 'en':
            input_dim = len(field.vocab)
            src_pad_idx = field.vocab.stoi[field.pad_token]
        elif name == 'ch':
            output_dim = len(field.vocab)
            trg_pad_idx = field.vocab[field.pad_token]

    enc_args_feeder = ModuleArgsFeeder(input_dim=input_dim, embedding_dim=256, hidden_dim=512,
                                       embedding_dropout=0.1, rnn_type='LSTM',
                                       num_layers=2, rnn_dropout=0.2)
    dec_args_feeder = ModuleArgsFeeder(input_dim=output_dim, embedding_dim=128, hidden_dim=256,
                                       embedding_dropout=0.1, rnn_type='LSTM',
                                       num_layers=2, rnn_dropout=0.2)
    return ArgsFeeder(enc_args_feeder, [dec_args_feeder],
                      batch_size=64, src_pad_idx=src_pad_idx, trg_pad_idx=trg_pad_idx,
                      optim_choice='Adam', learning_rate=0.003, decay_patience=0,
                      lr_decay_factor=0.9, valid_criterion='ACC', early_stopping_patience=1000,
                      total_epochs=100, report_interval=50, exp_num=exp_num, multi_task_ratio=0, data_container=data_container,
                      src_lang='en', trg_lang='pinyin_str', auxiliary_name=None, quiet_translate=True,
                      valid_out_path=f"experiments/exp{exp_num}/valid.out", test_out_path=f"experiments/exp{exp_num}/test.out")


def test_seq2seq(args_feeder):
    decoder_args_feeder = args_feeder.decoder_args_feeders[0]
    encoder = BasicEncoder(args_feeder)
    feed_forward_decoder = \
        BasicFeedForwardDecoder(args_feeder,
                                AdditiveAttention(encoder_hidden_dim=args_feeder.encoder_args_feeder.hidden_dim,
                                                  decoder_hidden_dim=decoder_args_feeder.hidden_dim), decoder_index=0)
    bridge_layer = BridgeLayer(encoder_hidden_dim=args_feeder.encoder_args_feeder.hidden_dim,
                               decoder_hidden_dim=decoder_args_feeder.hidden_dim,
                               num_of_states=2)
    decoder = GreedyDecoder(feed_forward_decoder, bridge_layer, device=args_feeder.device)
    model = Seq2Seq(args_feeder, encoder, decoder, teacher_forcing_ratio=0.8).to(args_feeder.device)
    return model


if __name__ == '__main__':
    set_reproducibility(seed=1234)
    
    try:
        # DICT pinyin-str
        dict_dataset = DICT['data_container']
        seq2seq_args_feeder = set_up_args(dict_dataset, exp_num=500)
        seq2seq_args_feeder.trg_lang = 'pinyin_str'
        test_model = test_seq2seq(seq2seq_args_feeder)
        test_trainer = Trainer(seq2seq_args_feeder, test_model)
        test_trainer.run(burning_epoch=0)
        test_trainer.best_model_output()
        # DICT pinyin-char
        seq2seq_args_feeder = set_up_args(dict_dataset, exp_num=501)
        seq2seq_args_feeder.trg_lang = "pinyin_char"
        test_model = test_seq2seq(seq2seq_args_feeder)
        test_trainer = Trainer(seq2seq_args_feeder, test_model)
        test_trainer.run(burning_epoch=0)
        test_trainer.best_model_output()

        # NEWS pinyin-str
        news_dataset = NEWS['data_container']
        seq2seq_args_feeder = set_up_args(news_dataset, exp_num=502)
        seq2seq_args_feeder.trg_lang = 'pinyin_str'
        test_model = test_seq2seq(seq2seq_args_feeder)
        test_trainer = Trainer(seq2seq_args_feeder, test_model)
        test_trainer.run(burning_epoch=0)
        test_trainer.best_model_output()
        # NEWS pinyin-char
        seq2seq_args_feeder = set_up_args(news_dataset, exp_num=503)
        seq2seq_args_feeder.trg_lang = 'pinyin_char'
        test_model = test_seq2seq(seq2seq_args_feeder)
        test_trainer = Trainer(seq2seq_args_feeder, test_model)
        test_trainer.run(burning_epoch=0)
        test_trainer.best_model_output()
    except KeyboardInterrupt:
        print("Exiting loop")

The current device for PyTorch is cuda
Seq2Seq(
  (encoder): BasicEncoder(
    (embedding): Sequential(
      (0): Embedding(30, 256)
      (1): Dropout(p=0.1, inplace=False)
    )
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.2, bidirectional=True)
  )
  (decoder): GreedyDecoder(
    (feed_forward_decoder): BasicFeedForwardDecoder(
      (attention): AdditiveAttention(
        (additive_mapping): Linear(in_features=1280, out_features=256, bias=True)
        (v): Linear(in_features=256, out_features=1, bias=False)
      )
      (embedding): Sequential(
        (0): Embedding(437, 128)
        (1): Dropout(p=0.1, inplace=False)
      )
      (rnn): LSTM(1152, 256, num_layers=2, dropout=0.2)
      (prediction): Sequential(
        (0): Linear(in_features=1408, out_features=437, bias=True)
        (1): LogSoftmax()
      )
    )
    (bridge_layer): BridgeLayer(
      (bridge_layer): ModuleList(
        (0): Linear(in_features=1024, out_features=256, bias=True)
        (1): Linear(in_f

[Epoch: 2][#examples: 34688/46620][#steps: 2000]
	Train Loss: 0.299 | Train PPL:   1.348 | lr: 2.700e-03
-----Val------
[VAL]: The number of correct predictions (aux-task (single)): 3993/5828
-----Tst------
[TEST]: The number of correct predictions (aux-task (single)): 4031/5828

---------------------------------------
[Epoch: 2][Validatiing...]
		 Better Valid Loss! (at least equal)
		 Better Valid Acc! (at least equal)
	 Early Stopping Patience: 1000/1000
	 Val. Loss: 0.453 | Val. Acc: 0.685 | Val. PPL:   1.574
	 BEST. Val. Loss: 0.453 | BEST. Val. Acc: 0.685 | Val. Loss: 0.453 | BEST. Val. Epoch: 2 | BEST. Val. Step: 2000
---------------------------------------

[Epoch: 2][#examples: 37888/46620][#steps: 2050]
	Train Loss: 0.300 | Train PPL:   1.349 | lr: 2.700e-03
[Epoch: 2][#examples: 41088/46620][#steps: 2100]
	Train Loss: 0.300 | Train PPL:   1.350 | lr: 2.700e-03
[Epoch: 2][#examples: 44288/46620][#steps: 2150]
	Train Loss: 0.300 | Train PPL:   1.350 | lr: 2.700e-03
[VAL]: The 

[VAL]: The number of correct predictions (aux-task (single)): 4121/5828
-----Tst------
[TEST]: The number of correct predictions (aux-task (single)): 4129/5828

---------------------------------------
[Epoch: 5][Validatiing...]
	 Early Stopping Patience: 998/1000
	 Val. Loss: 0.473 | Val. Acc: 0.707 | Val. PPL:   1.605
	 BEST. Val. Loss: 0.423 | BEST. Val. Acc: 0.711 | Val. Loss: 0.430 | BEST. Val. Epoch: 4 | BEST. Val. Step: 3645
---------------------------------------

[Epoch: 5][#examples: 25920/46620][#steps: 4050]
	Train Loss: 0.146 | Train PPL:   1.157 | lr: 1.968e-03
[Epoch: 5][#examples: 29120/46620][#steps: 4100]
	Train Loss: 0.147 | Train PPL:   1.159 | lr: 1.968e-03
[Epoch: 5][#examples: 32320/46620][#steps: 4150]
	Train Loss: 0.147 | Train PPL:   1.158 | lr: 1.968e-03
[Epoch: 5][#examples: 35520/46620][#steps: 4200]
	Train Loss: 0.148 | Train PPL:   1.160 | lr: 1.968e-03
[Epoch: 5][#examples: 38720/46620][#steps: 4250]
	Train Loss: 0.150 | Train PPL:   1.162 | lr: 1.968e-03

[Epoch: 8][#examples: 13952/46620][#steps: 6050]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 1.291e-03
[Epoch: 8][#examples: 17152/46620][#steps: 6100]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 1.291e-03
[Epoch: 8][#examples: 20352/46620][#steps: 6150]
	Train Loss: 0.048 | Train PPL:   1.049 | lr: 1.291e-03
[Epoch: 8][#examples: 23552/46620][#steps: 6200]
	Train Loss: 0.049 | Train PPL:   1.050 | lr: 1.291e-03
[Epoch: 8][#examples: 26752/46620][#steps: 6250]
	Train Loss: 0.048 | Train PPL:   1.050 | lr: 1.291e-03
[Epoch: 8][#examples: 29952/46620][#steps: 6300]
	Train Loss: 0.049 | Train PPL:   1.050 | lr: 1.291e-03
[Epoch: 8][#examples: 33152/46620][#steps: 6350]
	Train Loss: 0.049 | Train PPL:   1.050 | lr: 1.291e-03
[Epoch: 8][#examples: 36352/46620][#steps: 6400]
	Train Loss: 0.049 | Train PPL:   1.050 | lr: 1.291e-03
[Epoch: 8][#examples: 39552/46620][#steps: 6450]
	Train Loss: 0.050 | Train PPL:   1.051 | lr: 1.291e-03
[Epoch: 8][#examples: 42752/46620][#steps: 6500]
	Train

[Epoch: 11][#examples: 8384/46620][#steps: 8150]
	Train Loss: 0.018 | Train PPL:   1.018 | lr: 9.414e-04
[Epoch: 11][#examples: 11584/46620][#steps: 8200]
	Train Loss: 0.017 | Train PPL:   1.017 | lr: 9.414e-04
[Epoch: 11][#examples: 14784/46620][#steps: 8250]
	Train Loss: 0.017 | Train PPL:   1.017 | lr: 9.414e-04
[Epoch: 11][#examples: 17984/46620][#steps: 8300]
	Train Loss: 0.017 | Train PPL:   1.017 | lr: 9.414e-04
[Epoch: 11][#examples: 21184/46620][#steps: 8350]
	Train Loss: 0.017 | Train PPL:   1.017 | lr: 9.414e-04
[Epoch: 11][#examples: 24384/46620][#steps: 8400]
	Train Loss: 0.017 | Train PPL:   1.017 | lr: 9.414e-04
[Epoch: 11][#examples: 27584/46620][#steps: 8450]
	Train Loss: 0.017 | Train PPL:   1.017 | lr: 9.414e-04
[Epoch: 11][#examples: 30784/46620][#steps: 8500]
	Train Loss: 0.017 | Train PPL:   1.017 | lr: 9.414e-04
-----Val------
[VAL]: The number of correct predictions (aux-task (single)): 4256/5828
-----Tst------
[TEST]: The number of correct predictions (aux-task

...

[Epoch: 92][#examples: 42240/81252][#steps: 117500]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 9.838e-08
-----Val------
[VAL]: The number of correct predictions (aux-task (single)): 317/513
-----Tst------
[TEST]: The number of correct predictions (aux-task (single)): 679/1000

---------------------------------------
[Epoch: 92][Validatiing...]
	 Early Stopping Patience: 691/1000
	 Val. Loss: 1.567 | Val. Acc: 0.618 | Val. PPL:   4.794
	 BEST. Val. Loss: 1.108 | BEST. Val. Acc: 0.626 | Val. Loss: 1.539 | BEST. Val. Epoch: 16 | BEST. Val. Step: 21590
---------------------------------------

[Epoch: 92][#examples: 45440/81252][#steps: 117550]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 92][#examples: 48640/81252][#steps: 117600]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 92][#examples: 51840/81252][#steps: 117650]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 92][#examples: 55040/81252][#steps: 117700]
	Train Loss: 0.047 | Trai

[Epoch: 94][#examples: 17280/81252][#steps: 119650]
	Train Loss: 0.049 | Train PPL:   1.050 | lr: 9.838e-08
[Epoch: 94][#examples: 20480/81252][#steps: 119700]
	Train Loss: 0.050 | Train PPL:   1.051 | lr: 9.838e-08
[Epoch: 94][#examples: 23680/81252][#steps: 119750]
	Train Loss: 0.049 | Train PPL:   1.050 | lr: 9.838e-08
[Epoch: 94][#examples: 26880/81252][#steps: 119800]
	Train Loss: 0.050 | Train PPL:   1.051 | lr: 9.838e-08
[Epoch: 94][#examples: 30080/81252][#steps: 119850]
	Train Loss: 0.049 | Train PPL:   1.050 | lr: 9.838e-08
[Epoch: 94][#examples: 33280/81252][#steps: 119900]
	Train Loss: 0.049 | Train PPL:   1.050 | lr: 9.838e-08
[Epoch: 94][#examples: 36480/81252][#steps: 119950]
	Train Loss: 0.048 | Train PPL:   1.050 | lr: 9.838e-08
[Epoch: 94][#examples: 39680/81252][#steps: 120000]
	Train Loss: 0.048 | Train PPL:   1.049 | lr: 9.838e-08
-----Val------
[VAL]: The number of correct predictions (aux-task (single)): 317/513
-----Tst------
[TEST]: The number of correct predic

[VAL]: The number of correct predictions (aux-task (single)): 317/513
-----Tst------
[TEST]: The number of correct predictions (aux-task (single)): 679/1000

---------------------------------------
[Epoch: 96][Validatiing...]
	 Early Stopping Patience: 678/1000
	 Val. Loss: 1.567 | Val. Acc: 0.618 | Val. PPL:   4.793
	 BEST. Val. Loss: 1.108 | BEST. Val. Acc: 0.626 | Val. Loss: 1.539 | BEST. Val. Epoch: 16 | BEST. Val. Step: 21590
---------------------------------------

[Epoch: 96][#examples: 8320/81252][#steps: 122050]
	Train Loss: 0.048 | Train PPL:   1.049 | lr: 9.838e-08
[Epoch: 96][#examples: 11520/81252][#steps: 122100]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 96][#examples: 14720/81252][#steps: 122150]
	Train Loss: 0.049 | Train PPL:   1.050 | lr: 9.838e-08
[Epoch: 96][#examples: 17920/81252][#steps: 122200]
	Train Loss: 0.048 | Train PPL:   1.050 | lr: 9.838e-08
[Epoch: 96][#examples: 21120/81252][#steps: 122250]
	Train Loss: 0.049 | Train PPL:   1.050 |

[VAL]: The number of correct predictions (aux-task (single)): 317/513

---------------------------------------
[Epoch: 97][Validatiing...]
	 Early Stopping Patience: 672/1000
	 Val. Loss: 1.567 | Val. Acc: 0.618 | Val. PPL:   4.793
	 BEST. Val. Loss: 1.108 | BEST. Val. Acc: 0.626 | Val. Loss: 1.539 | BEST. Val. Epoch: 16 | BEST. Val. Step: 21590
---------------------------------------

Epoch: 98 | Time: 2m 2s
	Train Loss: 0.046 | Train PPL:   1.047
	 Val. Loss: 1.567 | Val. Acc: 0.618 | Val. PPL:   4.793
[Train]: Current Teacher Forcing Ratio: 0.200
[Epoch: 98][#examples: 2560/81252][#steps: 124500]
	Train Loss: 0.036 | Train PPL:   1.037 | lr: 9.838e-08
-----Val------
[VAL]: The number of correct predictions (aux-task (single)): 317/513
-----Tst------
[TEST]: The number of correct predictions (aux-task (single)): 679/1000

---------------------------------------
[Epoch: 98][Validatiing...]
	 Early Stopping Patience: 671/1000
	 Val. Loss: 1.567 | Val. Acc: 0.618 | Val. PPL:   4.793
	 B

[Epoch: 99][#examples: 58880/81252][#steps: 126650]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 99][#examples: 62080/81252][#steps: 126700]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 99][#examples: 65280/81252][#steps: 126750]
	Train Loss: 0.046 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 99][#examples: 68480/81252][#steps: 126800]
	Train Loss: 0.046 | Train PPL:   1.047 | lr: 9.838e-08
[Epoch: 99][#examples: 71680/81252][#steps: 126850]
	Train Loss: 0.046 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 99][#examples: 74880/81252][#steps: 126900]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 99][#examples: 78080/81252][#steps: 126950]
	Train Loss: 0.046 | Train PPL:   1.048 | lr: 9.838e-08
[Epoch: 99][#examples: 81280/81252][#steps: 127000]
	Train Loss: 0.046 | Train PPL:   1.048 | lr: 9.838e-08
-----Val------
[VAL]: The number of correct predictions (aux-task (single)): 317/513
-----Tst------
[TEST]: The number of correct predic